In [ ]:
import requests
from bs4 import BeautifulSoup

from openpyxl import Workbook
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#obter lista de cidades de Portugal

page_cidades_rec = requests.get("https://www.numbeo.com/cost-of-living/country_result.jsp?country=Portugal")
    
soup = BeautifulSoup(page_cidades_rec.content, 'html.parser')

cidade_rec_var = [x.text for x in soup.find_all('select', {'id': 'city'})]

lista_cidade_temp = [i.split('\n') for i in cidade_rec_var]
del lista_cidade_temp[0][0:2]
del lista_cidade_temp[0][-1]
lista_cidades_base = lista_cidade_temp[0]
#print(lista_cidades)

#substitui ' ' por '-'
lista_cidades = []
for string in lista_cidades_base:
  new_string = string.replace(" ", "-") #Modify old string.
  lista_cidades.append(new_string) #Add new string to list.
print(lista_cidades)




['Albufeira', 'Alcabideche', 'Algarve', 'Alges', 'Almada', 'Alverca-do-Ribatejo', 'Amadora', 'Amarante', 'Amora', 'Angra-do-Heroismo', 'Aveiro', 'Barcelos', 'Barreiro', 'Beja', 'Braga', 'Braganca', 'Caldas-da-Rainha', 'Cascais', 'Castelo-Branco', 'Chaves', 'Coimbra', 'Covilha', 'Ericeira', 'Ermesinde', 'Evora', 'Faro', 'Felgueiras', 'Figueira-da-Foz', 'Funchal', 'Guarda', 'Guimaraes', 'Horta', 'Lagos', 'Leiria', 'Lisbon', 'Loures', 'Lousã', 'Maia', 'Matosinhos', 'Montijo', 'Odivelas', 'Oeiras', 'Oliveira-de-Azemeis', 'Penafiel', 'Peniche', 'Pinhal-Novo', 'Ponta-Delgada', 'Portalegre', 'Portimao', 'Porto', 'Queluz', 'Rio-Tinto', 'Rio-de-Mouro', 'Santarem', 'Sao-Joao-da-Madeira', 'Seixal', 'Sesimbra', 'Setubal', 'Sines', 'Tavira', 'Tomar', 'Torres-Vedras', 'Viana-do-Castelo', 'Vila-Franca-de-Xira', 'Vila-Nova-de-Gaia', 'Vila-Real', 'Vila-do-Conde', 'Viseu']


Loop de inserção de dados das cidades.


In [ ]:
#para url do tipo: https://www.numbeo.com/cost-of-living/in/Braga

#lista_cidades = ["Braga", "Porto", "Lisbon", "Faro"]


for i in range(len(lista_cidades_base)):
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s?displayCurrency=EUR" % lista_cidades_base[i])
    
  page_cidades

  soup = BeautifulSoup(page_cidades.content, 'html.parser')

  z = soup.find_all('td', class_="priceValue")
  list(z)

  
  lista_val = []

  print(lista_cidades_base[i])

  lista_val.append(lista_cidades_base[i])
  for x in range(len(z)):
    
    lista_val.append(z[x].get_text())
    
  ## excel append section ##

  #path to the excel file
  excel_file_c1 = '/content/drive/My Drive/slotMachDatasets/Cidades_teste1.xlsx'

  #load excel file
  wb_c1 = load_workbook(excel_file_c1)

  #declares the active sheet were to append
  f1 = wb_c1.active
  #appends values to declared sheet
  f1.append(lista_val)

  #saves changes made to the excel file
  wb_c1.save(filename=excel_file_c1)

In [ ]:
#para url do tipo: https://www.numbeo.com/cost-of-living/in/Braga com "-"

#lista_cidades = ["Braga", "Porto", "Lisbon", "Faro"]


for i in range(len(lista_cidades)):
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s&displayCurrency=EUR" % lista_cidades[i])
    
  page_cidades

  soup = BeautifulSoup(page_cidades.content, 'html.parser')

  z = soup.find_all('td', class_="priceValue")
  list(z)

  
  lista_val = []

  print(lista_cidades[i])

  lista_val.append(lista_cidades[i])
  for x in range(len(z)):
    
    lista_val.append(z[x].get_text())
    
  ## excel append section ##

  #path to the excel file
  excel_file_c1 = '/content/drive/My Drive/slotMachDatasets/Cidades_teste1.xlsx'

  #load excel file
  wb_c1 = load_workbook(excel_file_c1)

  #declares the active sheet were to append
  f1 = wb_c1.active
  #appends values to declared sheet
  f1.append(lista_val)

  #saves changes made to the excel file
  wb_c1.save(filename=excel_file_c1)

In [ ]:
#para url do tipo: https://www.numbeo.com/cost-of-living/in/Albufeira-Portugal


for i in range(len(lista_cidades)):
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR" % lista_cidades[i])
    
  page_cidades

  soup = BeautifulSoup(page_cidades.content, 'html.parser')

  z = soup.find_all('td', class_="priceValue")
  list(z)

  
  lista_val = []

  print(lista_cidades[i])

  lista_val.append(lista_cidades[i])
  for x in range(len(z)):
    
    lista_val.append(z[x].get_text())
    
  ## excel append section ##

  #path to the excel file
  excel_file_c1 = '/content/drive/My Drive/slotMachDatasets/Cidades_teste1.xlsx'

  #load excel file
  wb_c1 = load_workbook(excel_file_c1)

  #declares the active sheet were to append
  f1 = wb_c1.active
  #appends values to declared sheet
  f1.append(lista_val)

  #saves changes made to the excel file
  wb_c1.save(filename=excel_file_c1)

In [ ]:
#substitui 'de' por 'De' e "do" por "Do"
lista_cidades_De = []
for string in lista_cidades:
  new_string = string.replace("de", "De") #Modify old string.
  lista_cidades_De.append(new_string) #Add new string to list.
print(lista_cidades_De)

lista_cidades_Do = []
for string in lista_cidades:
  new_string = string.replace("do", "Do") #Modify old string.
  lista_cidades_Do.append(new_string) #Add new string to list.
print(lista_cidades_Do)

In [ ]:
#para url do tipo: contem De


for i in range(len(lista_cidades_De)):
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR" % lista_cidades_De[i])
    
  page_cidades

  soup = BeautifulSoup(page_cidades.content, 'html.parser')

  z = soup.find_all('td', class_="priceValue")
  list(z)

  
  lista_val = []

  print(lista_cidades_De[i])

  lista_val.append(lista_cidades_De[i])
  for x in range(len(z)):
    
    lista_val.append(z[x].get_text())
    
  ## excel append section ##

  #path to the excel file
  excel_file_c1 = '/content/drive/My Drive/slotMachDatasets/Cidades_teste1.xlsx'

  #load excel file
  wb_c1 = load_workbook(excel_file_c1)

  #declares the active sheet were to append
  f1 = wb_c1.active
  #appends values to declared sheet
  f1.append(lista_val)

  #saves changes made to the excel file
  wb_c1.save(filename=excel_file_c1)

In [ ]:
#para url do tipo: contem Do


for i in range(len(lista_cidades_Do)):
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR" % lista_cidades_Do[i])
    
  page_cidades

  soup = BeautifulSoup(page_cidades.content, 'html.parser')

  z = soup.find_all('td', class_="priceValue")
  list(z)

  
  lista_val = []

  print(lista_cidades_Do[i])

  lista_val.append(lista_cidades_Do[i])
  for x in range(len(z)):
    
    lista_val.append(z[x].get_text())
    
  ## excel append section ##

  #path to the excel file
  excel_file_c1 = '/content/drive/My Drive/slotMachDatasets/Cidades_teste1.xlsx'

  #load excel file
  wb_c1 = load_workbook(excel_file_c1)

  #declares the active sheet were to append
  f1 = wb_c1.active
  #appends values to declared sheet
  f1.append(lista_val)

  #saves changes made to the excel file
  wb_c1.save(filename=excel_file_c1)

In [ ]:

#https://www.numbeo.com/cost-of-living/in/Viana-do-Castelo?displayCurrency=EUR

#para url do tipo: viana do castelo


for i in range(len(lista_cidades_Do)):
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/Viana-do-Castelo?displayCurrency=EUR")
    
  page_cidades

  soup = BeautifulSoup(page_cidades.content, 'html.parser')

  z = soup.find_all('td', class_="priceValue")
  list(z)

  
  lista_val = []

  print(lista_cidades_Do[i])

  lista_val.append(lista_cidades_Do[i])
  for x in range(len(z)):
    
    lista_val.append(z[x].get_text())
    
  ## excel append section ##

  #path to the excel file
  excel_file_c1 = '/content/drive/My Drive/slotMachDatasets/Cidades_teste1.xlsx'

  #load excel file
  wb_c1 = load_workbook(excel_file_c1)

  #declares the active sheet were to append
  f1 = wb_c1.active
  #appends values to declared sheet
  f1.append(lista_val)

  #saves changes made to the excel file
  wb_c1.save(filename=excel_file_c1)